In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI= "sqlite:///mlruns.db"

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [4]:
client.search_experiments()

[<Experiment: artifact_location='/home/jithish/Documents/zoomcamp/03/mlruns/0', creation_time=1706010704804, experiment_id='0', last_update_time=1706010704804, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
client.create_experiment(name= "test-experiment")   

'1'

In [11]:


runs = client.search_runs(experiment_ids=0, max_results=3, order_by=["metrics.rmse ASC"])

In [13]:
for run in runs:
    print(run.info.run_id, run.data.metrics["rmse"])

c32c1501a6684646a02f166c27cb35f0 6.3371229791404255
1190bccc40cc472994c45a65c085f004 6.3371229791404255
2a9b5deb6981404db4d1c5cec285cb37 6.3371229791404255


In [14]:
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [41]:
run_id= "c32c1501a6684646a02f166c27cb35f0"
model_uri = "runs:/{run_id}/model".format(run_id=run_id)

mlflow.register_model(model_uri, "model-1")

Registered model 'model-1' already exists. Creating a new version of this model...
Created version '2' of model 'model-1'.


<ModelVersion: aliases=[], creation_timestamp=1706125757925, current_stage='None', description=None, last_updated_timestamp=1706125757925, name='model-1', run_id='c32c1501a6684646a02f166c27cb35f0', run_link=None, source='/home/jithish/Documents/zoomcamp/03/mlruns/0/c32c1501a6684646a02f166c27cb35f0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [18]:
model_name = "model-1"
versions= client.get_latest_versions(name=model_name)

for version in versions:
    print(f"version: {version.version}, stage: {version.current_stage}")
    

version: 1, stage: Staging


/tmp/ipykernel_44164/72295715.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  versions= client.get_latest_versions(name=model_name)


In [19]:
client.transition_model_version_stage(\
    name=model_name, version=1, stage="Production")

/tmp/ipykernel_44164/159222312.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(\


<ModelVersion: aliases=[], creation_timestamp=1706111229552, current_stage='Production', description=None, last_updated_timestamp=1706123752203, name='model-1', run_id='c32c1501a6684646a02f166c27cb35f0', run_link=None, source='/home/jithish/Documents/zoomcamp/03/mlruns/0/c32c1501a6684646a02f166c27cb35f0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [53]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'].astype(str) + '_' + df['DOLocationID'].astype(str)
    categorical =['PU_DO']
    numerical = ['trip_distance']
    train_dicts= df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(X_test, y_test, logged_model):
    model = mlflow.pyfunc.load_model(logged_model)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"rmse: {rmse}")
    return rmse

In [38]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [24]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-03-01 00:05:42,2021-03-01 00:14:03,N,1.0,83,129,1.0,1.56,7.50,0.50,0.5,0.00,0.0,None,0.3,8.80,1.0,1.0,0.0
1,2,2021-03-01 00:21:03,2021-03-01 00:26:17,N,1.0,243,235,1.0,0.96,6.00,0.50,0.5,0.00,0.0,None,0.3,7.30,2.0,1.0,0.0
2,2,2021-03-01 00:02:06,2021-03-01 00:22:26,N,1.0,75,242,1.0,9.93,28.00,0.50,0.5,2.00,0.0,None,0.3,31.30,1.0,1.0,0.0
3,2,2021-03-01 00:24:03,2021-03-01 00:31:43,N,1.0,242,208,1.0,2.57,9.50,0.50,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.0
4,1,2021-03-01 00:11:10,2021-03-01 00:14:46,N,1.0,41,151,1.0,0.80,5.00,0.50,0.5,1.85,0.0,None,0.3,8.15,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83822,2,2021-03-31 22:07:00,2021-03-31 22:13:00,None,NaN,41,75,NaN,1.48,8.46,0.00,0.0,1.44,0.0,None,0.3,10.20,NaN,NaN,NaN
83823,2,2021-03-31 22:56:00,2021-03-31 23:13:00,None,NaN,95,95,NaN,0.09,54.25,2.75,0.0,0.00,0.0,None,0.3,57.30,NaN,NaN,NaN
83824,2,2021-03-31 22:36:00,2021-03-31 22:45:00,None,NaN,95,95,NaN,0.66,8.11,0.00,0.0,0.00,0.0,None,0.3,8.41,NaN,NaN,NaN
83825,2,2021-03-31 23:35:00,2021-04-01 00:00:00,None,NaN,37,14,NaN,9.58,36.83,2.75,0.0,0.00,0.0,None,0.3,39.88,NaN,NaN,NaN


In [50]:
run_id= "c32c1501a6684646a02f166c27cb35f0"

client.download_artifacts(run_id= run_id, path= "preprocessor", dst_path= '.')


'/home/jithish/Documents/zoomcamp/03/preprocessor'

In [43]:
import pickle

with open("preprocessor/preprocessor.bin", "rb") as f_in:
    dv = pickle.load(f_in)

In [44]:
X_test = preprocess(df, dv)

In [45]:
target = "duration"
y_test= df[target].values

In [54]:
logged_model = 'runs:/c32c1501a6684646a02f166c27cb35f0/models_mlflow'
test_model(X_test, y_test, logged_model)

/home/jithish/.cache/pypoetry/virtualenvs/zoomcamp-LVGFZID_-py3.11/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [03:56:37] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


rmse: 6.282885318450371


6.282885318450371